<img style="float: center;" src='./assets/jWST-SS-1300x300_Banner2.jpg' width="1000px"/> 

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from astropy.table import Table
import os,shutil,glob,sys,traceback
from astropy.cosmology import LambdaCDM
import pdb
import pickle
from astropy.io import fits
from astroquery.mast import Observations
import jhat
from jhat import jwst_photclass,hst_photclass,st_wcs_align
import space_phot
import astropy
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.time import Time
from jwst.pipeline import Image3Pipeline
import pprint
from astroquery.mast import Observations
import subprocess
import sewpy
from astropy.wcs import WCS
import requests
import os
import re

# Summer School Week 2
## Difference Imaging

### Author: David Coulter (JHU/STScI), Justin Pierel (STScI), Mike Engesser (STScI)

**Purpose**:<BR>
This notebook will cover a basic workflow for difference imaging for the purpose of discovering transients, and to enable photometry measurements on the resulting sources.

**Data**:<BR>
This example is set up to use an example dataset from the JADES program, PID 1180. Please see the JADES data download notebook for the astroquery code to download the specific files for this exercise from MAST, or you can retrieve the data from Box: 


**Additional Dependency Install**:<BR>
We need to install both Source Extractor (SE) and Sewpy, a Python wrapper for SE. We use conda-forge to install a pre-compiled SE library, and then install Sewpy from source.<BR>

[Sewpy](https://sewpy.readthedocs.io/en/latest/installation.html)<br>
[SExtractor](https://sextractor.readthedocs.io/en/latest/Using.html)
Better [Documentation](https://astro.dur.ac.uk/~pdraper/extractor/Guide2source_extractor.pdf)

In [ ]:
!conda install -y conda-forge::astromatic-source-extractor
!pip install git+https://github.com/megalut/sewpy

<hr style="border:1px solid gray"> </hr>

# Section 1: Aligning Lvl2 Cals and Preparing to Build Mosaics

### Inputs: 
* Raw Lvl 2 Cals for each epoch
   * F200W, F277W
* Reference catalog for alignment in the field
   * JADES [HLSP](https://jades-survey.github.io/)
### Outputs:
* Aligned Lvl 2 Cals 

In [ ]:
# Get supernova positions
# https://www.wis-tns.org/object/2023adta
SN2023adta = SkyCoord(53.135283, -27.814517, unit=(u.deg, u.deg), frame='icrs')

# https://www.wis-tns.org/object/2023adsy
SN2023adsy = SkyCoord(53.134853, -27.820899, unit=(u.deg, u.deg), frame='icrs')

### For convenience, let's arrange files into difference epochs to make processing more straight forward.

In [ ]:
# Get epoch 1 and epoch 2 cals in an array
epoch1_F356W_files = glob.glob("./JADES_data/epoch1/*nrcblong_cal.fits")
epoch1_F200W_files = glob.glob("./JADES_data/epoch1/*nrcb2_cal.fits")
epoch1_files = [] + epoch1_F356W_files + epoch1_F200W_files

epoch2_F356W_files = glob.glob("./JADES_data/epoch2/*nrcblong_cal.fits")
epoch2_F200W_files = glob.glob("./JADES_data/epoch2/*nrcb2_cal.fits")
epoch2_files = [] + epoch2_F356W_files + epoch2_F200W_files

# Make output dirs
jhat_epoch1_outsubdir = "epoch1_aligned"
jhat_epoch2_outsubdir = "epoch2_aligned"
os.makedirs(jhat_epoch1_outsubdir, exist_ok=True)
os.makedirs(jhat_epoch2_outsubdir, exist_ok=True)

### Now, let's grab the JADES photometric catalog -- we will use this for alignment and for passing sources to our convolution ([HOTPANTS.](https://github.com/acbecker/hotpants))

In [ ]:
catalog_url = "https://archive.stsci.edu/hlsps/jades/dr2/goods-s/catalogs/hlsp_jades_jwst_nircam_goods-s-deep_photometry_v2.0_catalog.fits"
output_dir = "./JADES_data"
filename = catalog_url.split('/')[-1]

# Create the directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

file_path = os.path.join(output_dir, filename)

r = requests.get(catalog_url, stream=True)
with open(file_path, 'wb') as f:
    for chunk in r.iter_content(chunk_size=1024):
        if chunk:
            f.write(chunk)

### Build the catalog by converting fluxes to mags. See catalog documentation [here.](https://archive.stsci.edu/hlsps/jades/hlsp_jades_jwst_nircam_goods-s-deep_photometry_v2.0_catalog-ext-readme.pdf)

In [ ]:
# Make a magnitude ranked input catalog for JHAT from the JADES photometric catalog we just downloaded.
ref_fits = "./JADES_data/hlsp_jades_jwst_nircam_goods-s-deep_photometry_v2.0_catalog.fits"
ref_catname = "./JADES_data/jhat_cat.txt"
max_depth = 28.0
flux_units = astropy.units.nJy

with fits.open(ref_fits, memmap=True) as cat_file:

    table = Table(cat_file["CIRC_BSUB"].data)

    filt_name = "F150W"
    flux_key = "{filt}_CIRC0".format(filt=filt_name)
    err_key = "{filt}_CIRC0_e".format(filt=filt_name)

    src_id = table["ID"]
    ra = table["RA"]
    dec = table["DEC"]
    nJy = table[flux_key]
    nJy_err = table[err_key]

    flux = nJy*flux_units
    flux_err = nJy_err*flux_units
    mag = flux.to(astropy.units.ABmag).value
    mag_err = flux_err.to(astropy.units.ABmag).value
    
    output_tbl = Table([src_id, ra, dec, mag, mag_err], names=["ID", "RA", "DEC", "MAG", "MAG_ERR"])

    output_tbl = output_tbl[output_tbl["MAG"] < max_depth]
    output_tbl.write(ref_catname, format="ascii", overwrite=True)

In [ ]:
# This is a utility function to allow us to visualize things in DS9
def create_pixregionfile(x,y,regionname,color,coords='image',radius=1):
    if isinstance(radius,int):
        radius = [radius]*len(x)
    with open(regionname, 'w') as f:
        if isinstance(color,str):
            f.write('global color={0} dashlist=8 3 width=2 font=\"helvetica 10 normal roman\" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 \n'.format(color))
            do_col = False
        else:
            do_col = True
            f.write('global dashlist=8 3 width=2 font=\"helvetica 10 normal roman\" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 \n')
        f.write('%s \n'%coords)
        for star in range(len(x)):
            xval = x[star]
            yval = y[star]
            if do_col:
                f.write('circle({ra},{dec},{radius}") # color={color}\n'.format(ra=xval, dec=yval,radius=radius[star],color=color[star]))
            else:
                f.write('circle({ra},{dec},{radius}")\n'.format(ra=xval, dec=yval,radius=radius[star]))
    f.close()

In [ ]:
# Function wrapper for JHAT
def align_and_create_region_files(input_file, output_subdir, reference_cat_name, max_depth):
    
    wcs_align = st_wcs_align()
    
    try:
        print("\t***** Processing File: `%s` *****" % f)
    
        wcs_align.run_all(input_file,
                      telescope='jwst',
                      outsubdir=output_subdir,
                      refcat_racol='RA',
                      refcat_deccol='DEC',
                      refcat_magcol='MAG',
                      refcat_magerrcol='MAG_ERR',
                      overwrite=True,
                      d2d_max=0.25,
                      showplots=0,
                      refcatname=reference_cat_name,
                      histocut_order='dxdy',
                      sharpness_lim=(0.3,0.9), # 0 to 1
                      roundness1_lim=(-0.7, 0.7), # -1 to 1
                      SNR_min= 3,
                      dmag_max=1.0,
                      objmag_lim =(14,max_depth),
                      saveplots=2,
                      savephottable=True)
    
        good_matches_file_name = os.path.basename(f).replace('_cal.fits', '.goodmatches.csv')
        good_matches_file_path = "./{base_path}/{file_name}".format(base_path=jhat_epoch1_outsubdir, file_name=good_matches_file_name)
        good_match_tbl = Table.read(good_matches_file_path, format="ascii")

        create_pixregionfile(good_match_tbl["RA"], 
                             good_match_tbl["DEC"],
                             good_matches_file_path.replace("csv", "reg"), 
                             color="red", coords="icrs", radius=[0.4]*len(good_match_tbl))
        
              
    except Exception as e:
        print("\t\t***** FAILED Processing File: `%s` *****" % f)
        print(e)

### OK, now we can invoke JHAT!

In [ ]:
# Do Epoch 1
e1_cnt = len(epoch1_files)
for i, f in enumerate(epoch1_files):
    curr = i+1
    print("\t***** Processing Epoch 1 File %s/%s: `%s` *****" % (curr, e1_cnt, f))
    align_and_create_region_files(f, jhat_epoch1_outsubdir, ref_catname, max_depth)

In [ ]:
# Do Epoch 2
e2_cnt = len(epoch2_files)
for i, f in enumerate(epoch2_files):
    curr = i+1
    print("\t***** Processing Epoch 2 File %s/%s: `%s` *****" % (curr, e2_cnt, f))
    align_and_create_region_files(f, jhat_epoch2_outsubdir, ref_catname, max_depth)

<hr style="border:1px solid gray"> </hr>

# Section 2: Building Aligned Lvl3 Mosaics And Projecting Them to the Same Tangential Plane

### Inputs: 
* Aligned Lvl 2 Cals for each epoch
   * F200W, F356W
### Outputs:
* 4 Mosiacs:
   * F200W Epoch1 (Reference Image), F200W Epoch2 (Science Image)
   * F356W Epoch1 (Reference Image), F356W Epoch2 (Science Image)

### We need to make a choice on the parameters of the location of the tangent plane. 

* Consider:
   * Where you want your central coordinate in RA/Dec space to be?
   * Where do you want to map that coordinate within the data array?

In [ ]:
aligned_epoch1_F356W_files = glob.glob("%s/*nrcblong_jhat.fits" % (jhat_epoch1_outsubdir))
aligned_epoch1_F200W_files = glob.glob("%s/*nrcb2_jhat.fits" % (jhat_epoch1_outsubdir))

aligned_epoch2_F356W_files = glob.glob("%s/*nrcblong_jhat.fits" % (jhat_epoch2_outsubdir))
aligned_epoch2_F200W_files = glob.glob("%s/*nrcb2_jhat.fits" % (jhat_epoch2_outsubdir))

# What will make up each of our mosaics?
pprint.pprint(aligned_epoch1_F356W_files)
pprint.pprint(aligned_epoch2_F200W_files)


# Choose an appropriate projection
sw_pix_scale = 30.0 / 1000 # milliarcseconds
lw_pix_scale = 60.0 / 1000 # milliarcseconds

# Positioned between both SNe
CRVAL1 = 53.1359270
CRVAL2 = -27.8174387

delta_ra = 0.01 # degrees
delta_dec = 0.01 # degrees
tile_shape = np.array((2048, 2048))

CRPIX1, CRPIX2 = tile_shape / 2
rotation = 0

### OK, now we can invoke the JWST pipeline!

In [ ]:
pipe3 = Image3Pipeline()

ep1_output_dir = "./epoch1_mosaic"
ep2_output_dir = "./epoch2_mosaic"
os.makedirs(ep1_output_dir, exist_ok=True)
os.makedirs(ep2_output_dir, exist_ok=True)

# We are setting parameters to be simple. However, what could you do to improve the mosaicing?
params = {
            'assign_mtwcs':         {'skip': True},
            'tweakreg':             {'skip': True},
            'outlier_detection':    {'skip': True},
            'source_catalog':       {'skip': True},
            'skymatch':             {'skip': True}
        }

params['resample'] = {  
                        'skip': False,
                        'pixfrac': 1.,
                        'kernel': 'square',
                        'fillval': 'indef',
                        'weight_type': 'ivm',
                        'in_memory': True,
                        'save_results': True
                    }

params['resample']['pixel_scale'] = sw_pix_scale  
params['resample']['rotation'] = rotation
params['resample']['output_shape'] = (int(tile_shape[0]), int(tile_shape[1]))

params['resample']['crpix'] = [CRPIX1, CRPIX2]
params['resample']['crval'] = [CRVAL1, CRVAL2]

# Perform SW first
f200w_epoch1_basefile = "f200w_epoch1"
f200w_epoch2_basefile = "f200w_epoch2"

pipe3.call([img for img in aligned_epoch1_F200W_files], 
           steps=params, 
           output_dir=ep1_output_dir, 
           output_file=f200w_epoch1_basefile, 
           save_results=True)

pipe3.call([img for img in aligned_epoch2_F200W_files], 
           steps=params, 
           output_dir=ep2_output_dir, 
           output_file=f200w_epoch2_basefile, 
           save_results=True)

# Perform LW next
f356w_epoch1_basefile = "f356w_epoch1"
f356w_epoch2_basefile = "f356w_epoch2"

params['resample']['pixel_scale'] = lw_pix_scale  

pipe3.call([img for img in aligned_epoch1_F356W_files], 
           steps=params, 
           output_dir=ep1_output_dir, 
           output_file=f356w_epoch1_basefile, 
           save_results=True)

pipe3.call([img for img in aligned_epoch2_F356W_files], 
           steps=params, 
           output_dir=ep2_output_dir, 
           output_file=f356w_epoch2_basefile, 
           save_results=True)


<hr style="border:1px solid gray"> </hr>

# Section 3: Image Subtraction Two Ways

### Inputs: 
* Mosaics for Epoch 1 and Epoch 2 in both filters
   * F200W, F356W
### Outputs:
* 2 Difference images via HOTPANTS:
   * F200W Epoch2 - Epoch1
   * F356W Epoch2 - Epoch1
* 2 Difference images via straight differencing:
   * F200W Epoch2 - Epoch1
   * F356W Epoch2 - Epoch1

**The following function uses [Docker](https://docs.docker.com/desktop/) to abstract away the need to install HOTPANTS**:<BR>
Docker is a virtualization framework that allows one to build very light weight containers that can execute Linux processes. Think of it like a very light weight virtual machine, and with it you can "version control" you computational environment.<BR>

**Note, there's also a utility function, `create_stamp_catalog`, which creates the second catalog we need for HOTPANTS**:<BR>
This source catalog is used for HOTPANTS to sample the PSFs of both files to allow it to convolve them.


In [ ]:
# This is a function that wraps a HOTPANTS call.
def run_hotpants(inst_name, filt_name, ref_input_file, sci_input_file, cat_file, curr_working_dir, diff_dir):

    # Utility dictionary to store the FWHM of the detectors in pixels.
    filters = {}
    filters['NIRCAM'] = {'F070W': 0.987, 'F090W': 1.103, 'F115W': 1.298, 'F140M': 1.553, 'F150W2': 1.628,
                         'F150W': 1.770,
                         'F162M': 1.801, 'F164N': 1.494, 'F182M': 1.990, 'F187N': 2.060, 'F200W': 2.141, 'F210M': 2.304,
                         'F212N': 2.341, 'F250M': 1.340, 'F277W': 1.444, 'F300M': 1.585, 'F322W2': 1.547,
                         'F323N': 1.711,
                         'F335M': 1.760, 'F356W': 1.830, 'F360M': 1.901, 'F405N': 2.165, 'F410M': 2.179, 'F430M': 2.300,
                         'F444W': 2.302, 'F460M': 2.459, 'F466N': 2.507, 'F470N': 2.535, 'F480M': 2.574}
    filters['NIRISS'] = {'F090W': 1.40, 'F115W': 1.40, 'F140M': 1.50, 'F150W': 1.50, 'F158M': 1.50, 'F200W': 1.50,
                         'F277W': 1.50, 'F356W': 1.60, 'F380M': 1.70, 'F430M': 1.80, 'F444W': 1.80, 'F480M': 1.80}
    filters['MIRI'] = {'F560W': 1.636, 'F770W': 2.187, 'F1000W': 2.888, 'F1130W': 3.318, 'F1280W': 3.713,
                       'F1500W': 4.354, 'F1800W': 5.224, 'F2100W': 5.989, 'F2550W': 7.312}
    filters['WFC3'] = {'F105W': 1.001, 'F110W': 1.019, 'F125W': 1.053, 'F140W': 1.100, 'F160W': 1.176}
    filters['ACS'] = {'F814W': 1.0}

    # Utility function to take multiextension FITS files and save out each extension as its own file. 
    # HOTPANTS requires single extension files.
    def unpack_fits(input_file, sci_out_filename, err_out_filename):
        
        # Unpack data into single extension files
        with fits.open(input_file, output_verify='fix') as dat:
    
            sci_data = dat['SCI', 1].data
            max_sci_val = np.nanmax(sci_data)
        
            err_data = dat['ERR', 1].data
            err_median = np.nanmedian(err_data)
            sci_pedastal = 50.0 * err_median
            print("\n\tMedian error value: `%0.4f`; Pedastal to add to Sci Im: `%0.4f`" % (err_median, sci_pedastal))
        
            dat[0].data = sci_data + sci_pedastal
            dat[0].header = dat[0].header + dat[1].header
            dat[0].writeto(sci_out_filename, overwrite=True)
        
            dat[0].data = err_data
            dat[0].writeto(err_out_filename, overwrite=True)
    
            return max_sci_val
    
    # Utility function to mask out regions with NaNs
    def create_mask(input_err_file, output_mask_file):

        with fits.open(input_err_file, output_verify='fix') as dat:
            sci_mask = np.zeros_like(dat[0].data)
            sci_mask[dat[0].data == 0] = 0x80
            sci_mask[np.isnan(dat[0].data)] = 0x80
        
            dat[0].data = sci_mask
            dat[0].scale('int16')
            dat.writeto(output_mask_file, overwrite=True)

    # Takes Tabular Fits file w/ specific columns... feel free to generalize
    def create_stamp_catalog(reference_cat_fits, input_file, filt_name, flux_thresh, stamp_outfile):
        
        # Build stampxy file
        with fits.open(reference_cat_fits, memmap=True) as cat_file:
        
            table = Table(cat_file["CIRC_BSUB"].data)            
            
            flux_key = "{filt}_CIRC0".format(filt=filt_name)
            err_key = "{filt}_CIRC0_e".format(filt=filt_name)
            
            ras = table["RA"]
            decs = table["DEC"]
        
            flux = table[flux_key]
            flux_err = table[err_key]

            # reverse sort so brightest sources first
            indices = (-flux).argsort()
        
            sorted_ra = ras[indices]
            sorted_dec = decs[indices]
            sorted_fluxes = flux[indices]
            sorted_flux_err = flux_err[indices]

            # get sources brighter than some threshold
            bright_ind = np.where(sorted_fluxes > flux_thresh)[0]
            bright_ra = sorted_ra[bright_ind]
            bright_dec = sorted_dec[bright_ind]
            bright_flux = sorted_fluxes[bright_ind]
            bright_flux_err = sorted_flux_err[bright_ind]
                 
            coords = SkyCoord(bright_ra, bright_dec, unit=(u.deg, u.deg))
            sci_obs3 = space_phot.observation3(input_file)
            y_max, x_max = sci_obs3.data.shape
            xs, ys = sci_obs3.wcs.world_to_pixel(coords)
            in_img_indices = np.where((xs >= 0) & (xs <= x_max) & (ys >= 0) & (ys <= y_max))[0]
            
            output_tbl = Table(
                [xs[in_img_indices],
                 ys[in_img_indices],
                 bright_ra[in_img_indices],
                 bright_dec[in_img_indices],
                 bright_flux[in_img_indices]], names=["X", "Y", "RA", "DEC", filt_name])
        
            output_tbl.write(stamp_outfile, format="ascii", overwrite=True)
            create_pixregionfile(xs[in_img_indices], ys[in_img_indices], stamp_outfile.replace("txt", "reg"), color="red",
                                 coords="image", radius=[0.4] * len(ys[in_img_indices]))

    
    # Create single extension file names for Science and Reference images
    sci_output_file = sci_input_file.replace(".fits", "_1.fits")
    sci_output_err_file = sci_input_file.replace(".fits", "_1.noise.fits")
    sci_output_mask_file = sci_input_file.replace(".fits", "_1.mask.fits")
    sci_stampxy_file = sci_input_file.replace('.fits', '.stampxy.txt')
    max_val_sci_im = unpack_fits(sci_input_file, sci_output_file, sci_output_err_file)

    ref_output_file = ref_input_file.replace(".fits", "_1.fits")
    ref_output_err_file = ref_input_file.replace(".fits", "_1.noise.fits")
    ref_output_mask_file = ref_input_file.replace(".fits", "_1.mask.fits")
    max_val_ref_im = unpack_fits(ref_input_file, ref_output_file, ref_output_err_file)

    # Create masks for Science and Reference images
    create_mask(sci_output_err_file, sci_output_mask_file)
    create_mask(ref_output_err_file, ref_output_mask_file)

    # Dymanically create the Difference Image file name
    diff_file_basename_formatter = "{filt_name}_{sci_epoch}_{ref_epoch}.diff.fits"
    epoch_regex = r'epoch\d+'

    # For convenience, I am assuming that "epoch" is in the file names... you would need to generalize if you have a different
    # naming convention
    match = re.search(epoch_regex, ref_input_file)
    ref_file_epoch = match.group(0)
    
    match = re.search(epoch_regex, sci_input_file)
    sci_file_epoch = match.group(0)

    # Set up the named files to capture HOTPANTS output
    diff_file = os.path.join(diff_dir, 
                             diff_file_basename_formatter.format(filt_name=filt_name, sci_epoch=sci_file_epoch, ref_epoch=ref_file_epoch))
    diff_err_file = diff_file.replace('.fits', '.noise.fits')
    diff_mask_file = diff_file.replace('.fits', '.mask.fits')
    diff_kernel_file = diff_file.replace('.fits', '.kernel.fits') 
    diff_stampxy_reg_file = diff_file.replace('.fits', '.stampxy.reg')

    # Build the catalog for HOTPANTS to sample the PSFs
    create_stamp_catalog(cat_file, sci_output_file, filt_name, flux_thresh=10.0, stamp_outfile=sci_stampxy_file)

    # A bit dodgey: We're assuming the same instrument and filter between science and template image
    sci_fwhm = 1.8
    if inst_name in filters:
        if filt_name in filters[inst_name]:
            sci_fwhm = filters[inst_name][filt_name]
    else:
        print(f'{inst_name} not found in {list(filters.keys())}, FWHM fixed to 1.8 pixels.')

    temp_fwhm = 1.8
    if inst_name in filters:
        if filt_name in filters[inst_name]:
            temp_fwhm = filters[inst_name][filt_name]
    else:
        print(f'{inst_name} not found in {list(filters.keys())}, FWHM fixed to 1.8 pixels.')


    # hotpants args
    ko = 2  # spatial order of kernel variation within region (2)
    bgo = 2  # spatial order of background variation within region (1)
    ssig = 3.0  # threshold for sigma clipping statistics  (3.0)
    
    ks = 2.0  # high sigma rejection for bad stamps in kernel fit (2.0)
    r = 2.5 * sci_fwhm  # FWHM * 2.5         # convolution kernel half width (10)
    kfm = 0.99  # fraction of abs(kernel) sum for ok pixel (0.990)
    
    il = 0  # lower valid data count, image (0)
    iu = max_val_sci_im  # dynamically take the highest in the science img        # upper valid data count, image (25000)
    iuk = iu  # set to 'iu' for now     # upper valid data count for kernel, image (iuthresh)
    
    tl = 0  # lower valid data count, template (0)
    tu = max_val_ref_im  # upper valid data count, template (25000)
    tuk = tu  # upper valid data count for kernel, template (tuthresh)
    
    nrx = 1 #1  # number of image regions in x dimension (1)
    nry = 1 #1  # number of image regions in y dimension (1)
    
    nsx = 8 #70  # number of each region's stamps in x dimension (10)
    nsy = 8 #90  # number of each region's stamps in y dimension (10)
    nss = 7  # number of centroids to use for each stamp (3) # DC: got a segmentation fault using "10"
    
    # ngauss degree0 sigma0 .. degreeN sigmaN]
    # : ngauss = number of gaussians which compose kernel (3)
    # : degree = degree of polynomial associated with gaussian #
    #            (6 4 2)
    # : sigma  = width of gaussian #
    #            (0.70 1.50 3.00)
    # : N = 0 .. ngauss - 1
    
    # : (3 6 0.70 4 1.50 2 3.00
    ng = (3, 6, sci_fwhm / 2.0, 4, sci_fwhm, 2, sci_fwhm * 2.0)
    
    rss = 2.5 * sci_fwhm  # 2.5 * FWHM scaled by the fwhm - radius of the substamp # half width substamp to extract around each centroid (15)
    ft = 5.0  # RMS threshold for good centroid in kernel fit (20.0)
    
    # -okn          # rescale noise for 'ok' pixels (0)
    c = "t"  # force convolution on (t)emplate or (i)mage (undef)
    n = "i"  # normalize to (t)emplate, (i)mage, or (u)nconvolved (t)
    # -sconv        # all regions convolved in same direction (0)
    
    afssc = 0  # autofind stamp centers so #=-nss when -ssf,-cmp (1)
    gridssc = 0
    
    fi = 0.0  # value for invalid (bad) pixels (1.0e-30)
    fin = 0.0  # noise image only fillvalue (0.0e+00)
    
    mins = 2.0  # Fraction of kernel half width to spread input mask (1.0)
    mous = 0.0  # Ditto output mask, negative = no diffim masking (1.0)
    
    v = 2  # level of verbosity, 0-2 (1)
    # -savexy = set file name . saves the X,Y positions of used, clipped, and all substamps in different colors
    # save as a reg file
    
    hotpants_arg = 'hotpants -inim {sci_file} -tmplim {temp_file} -outim {diff_file} -ini {sci_noise_im} \
                   -imi {sci_mask} -il {il} -iu {iu} -iuk {iuk} -tni {temp_noise_im} -tmi {temp_mask} \
                   -tl {tl} -tu {tu} -tuk {tuk} -nrx {nrx} -nry {nry} -nsx {nsx} -nsy {nsy} -nss {nss} -ng {ng_tuple} \
                   -rss {rss} -ft {ft} -r {r} -ko {ko} -bgo {bgo} -ssig {ssig} -ks {ks} -kfm {kfm} -okn -c {c} -n {n} -sconv \
                   -cmp {diff_substamps} -afssc {afssc} -gridssc {gridssc} -fi {fi} \
                   -oni {diff_noise_im}  -fin {fin} -mins {mins} \
                   -omi {diff_mask} -mous {mous} -oki {diff_kernel} -v {v} -savexy {savexy}'.format(
        sci_file=sci_output_file, temp_file=ref_output_file, diff_file=diff_file,
        sci_noise_im=sci_output_err_file, sci_mask=sci_output_mask_file, il=il, iu=iu, iuk=iuk, temp_noise_im=ref_output_err_file,
        temp_mask=ref_output_mask_file, tl=tl, tu=tu, tuk=tuk, nrx=nrx, nry=nry, nsx=nsx, nsy=nsy, nss=nss,
        ng_tuple=" ".join([str(_ng) for _ng in ng]),
        rss=rss, ft=ft, r=r, ko=ko, bgo=bgo, ssig=ssig, ks=ks, kfm=kfm, c=c, n=n, diff_substamps=sci_stampxy_file,
        afssc=afssc, gridssc=gridssc, fi=fi, diff_noise_im=diff_err_file, fin=fin,
        mins=mins, diff_mask=diff_mask_file, mous=mous, diff_kernel=diff_kernel_file, v=v, savexy=diff_stampxy_reg_file)
    
    print("Hotpants invocation:\n\t%s" % hotpants_arg)

    # This is where you need Docker!
    os.system("docker run --rm -v %s:/app ghcr.io/davecoulter/diffpype:1.0.3 %s" % (curr_working_dir, hotpants_arg))

    return diff_file


<img style="float: left;" src='./assets/hotpants.png' width="100px"/><h2>OK, now we can run HOTPANTS!</h2>


In [ ]:
# We need to map our current working path to the docker container to get output locally
local_working_dir = os.getcwd()

In [ ]:
# Create output dir
diff_outsubdir = "./diff"
os.makedirs(diff_outsubdir, exist_ok=True)

f200w_ref = "%s/%s_i2d.fits" % (ep1_output_dir, f200w_epoch1_basefile)
f200w_sci = "%s/%s_i2d.fits" % (ep2_output_dir, f200w_epoch2_basefile)

f200w_diff_im = run_hotpants(inst_name="NIRCAM", 
             filt_name="F200W", 
             ref_input_file=f200w_ref, 
             sci_input_file=f200w_sci, 
             cat_file=ref_fits, 
             curr_working_dir=local_working_dir,
             diff_dir=diff_outsubdir)


f356w_ref = "%s/%s_i2d.fits" % (ep1_output_dir, f356w_epoch1_basefile)
f356w_sci = "%s/%s_i2d.fits" % (ep2_output_dir, f356w_epoch2_basefile)

f356w_diff_im = run_hotpants(inst_name="NIRCAM", 
             filt_name="F356W", 
             ref_input_file=f356w_ref, 
             sci_input_file=f356w_sci, 
             cat_file=ref_fits, 
             curr_working_dir=local_working_dir,
             diff_dir=diff_outsubdir)

<img style="float: left;" src='./assets/haha.png' width="250px"/>
<b>Now let's do an alternative to HOTPANTS:</b><BR>
Because the PSF of JWST is so stable, it is possible to simple subtract the data arrays from each other, and estimating a resulting error image by adding the component error arrays in quadrature.<BR>

**This works for JWST but won't for everything!**<BR>
Can you think of some situations where the below approach would not work? Consider long baseline epochs, different position angles, different telescopes, ground based data, etc.


In [ ]:
# Run Straight Diff for F200W
straight_sci_f200w = fits.open(f200w_sci)
straight_ref_f200w = fits.open(f200w_ref)
straight_diff_f200w_img_ouput = f200w_diff_im.replace(".diff.fits", ".straight_diff.fits")
straight_diff_f200w_noise_ouput = straight_diff_f200w_img_ouput.replace(".fits", ".noise.fits")

straight_sci_f200w['SCI'].data -= straight_ref_f200w['SCI'].data
# Just using the `straight_sci_f200w` in-memory FITS file to write out to a new file
straight_sci_f200w.writeto(straight_diff_f200w_img_ouput, overwrite=True)

straight_sci_f200w['SCI'].data = np.sqrt(straight_ref_f200w['ERR'].data**2 + straight_sci_f200w['ERR'].data**2)
# Just using the `straight_sci_f200w` in-memory FITS file to write out to a new file
straight_sci_f200w.writeto(straight_diff_f200w_noise_ouput, overwrite=True)

# Run Straight Diff for F356W
straight_sci_f356w = fits.open(f356w_sci)
straight_ref_f356w = fits.open(f356w_ref)
straight_diff_f356w_img_ouput = f356w_diff_im.replace(".diff.fits", ".straight_diff.fits")
straight_diff_f356w_noise_ouput = straight_diff_f356w_img_ouput.replace(".fits", ".noise.fits")

straight_sci_f356w['SCI'].data -= straight_ref_f356w['SCI'].data
# Just using the `straight_sci_f356w` in-memory FITS file to write out to a new file
straight_sci_f356w.writeto(straight_diff_f356w_img_ouput, overwrite=True)

straight_sci_f356w['SCI'].data = np.sqrt(straight_ref_f356w['ERR'].data**2 + straight_sci_f356w['ERR'].data**2)
# Just using the `straight_sci_f356w` in-memory FITS file to write out to a new file
straight_sci_f356w.writeto(straight_diff_f356w_noise_ouput, overwrite=True)

<img style="float: left;" src='./assets/clippy.jpg' width="100px"/><b>Let's check our handiwork:</b>
* Open our Epoch1, Epoch2, and HOTPANTS Diff Image, and "Straight Diff" images
* Look at the noise properties (DS9 region statistics)
   * We want as little structure in the noise as possible
   * Zoom into a galaxy -- at best, complex structures should be removed with little residuals
   * Can you see any new sources by eye?

<hr style="border:1px solid gray"> </hr>

# Section 4: Source Extraction

### Inputs: 
* Difference images for both filters
   * F200W, F356W
### Outputs:
* 2 source catalogs with plottable regions

### Let's use a Python wrapper for Source Extractor

* Note:
   * This is not necessarily an optimum usage. What other parameters can we pass to Source Extractor?
   * Similarly, what other parameters can we measure?

In [ ]:
# Utility function to wrap source extraction
def run_sextractor_on_diff(diff_file, output_dir, snr_thresh):
    
    # class_star_cut = 0 # You can use this if you have a good stellar model
    sew = sewpy.SEW(params=["X_IMAGE", "Y_IMAGE", "FLUX_RADIUS(3)", "FLAGS", "XPEAK_WORLD", "YPEAK_WORLD","CLASS_STAR"],
                        config={"DETECT_MINAREA":3, "PHOT_FLUXFRAC":"0.3, 0.5, 0.8",'DETECT_THRESH':snr_thresh,
                                'BACKPHOTO_TYPE':'local'}, loglevel=0)
    
    diff_wcs = WCS(fits.open(diff_file)['SCI'])
    
    found = sew(diff_file)['table']
    found['xcentroid'] = found['X_IMAGE'] -1 # zero based to one based array fix
    found['ycentroid'] = found['Y_IMAGE'] -1

    # pdb.set_trace()
    found['skycoord_peak'] = diff_wcs.pixel_to_world(found['xcentroid'],found['ycentroid'])
    # found = found[found['CLASS_STAR']>class_star_cut]
    
    table_cols = ['source_id','ra','dec']
    table_data = [
        ['src_'+str(x) for x in np.arange(0,len(found),1)],
        [x.ra.value for x in found['skycoord_peak']],
        [x.dec.value for x in found['skycoord_peak']]
    ]
    
    source_cat = Table(names=table_cols,dtype=[str]+[float]+[float])
    
    for data_row in zip(*table_data):
        new_row = {}
        for i, d in enumerate(data_row):
            new_row[table_cols[i]] = d
        source_cat.add_row(new_row)
    
    source_cat.write(diff_file.replace(".fits", ".sources.txt"), format='ascii', overwrite=True)
    create_pixregionfile(source_cat['ra'], source_cat['dec'], diff_file.replace(".fits", ".sources.reg"), color="red",
                                     coords="icrs", radius=[0.4] * len(source_cat))    

### Let's invoke Source Extractor!

In [ ]:
sewpy_snr_thresh = 3.0
run_sextractor_on_diff(f200w_diff_im, diff_outsubdir, sewpy_snr_thresh)
run_sextractor_on_diff(f356w_diff_im, diff_outsubdir, sewpy_snr_thresh)

<img style="float: left;" src='./assets/clippy.jpg' width="100px"/><b>Let's check our handiwork:</b>
* Open our Epoch1, Epoch2, and HOTPANTS Diff Image, and "Straight Diff" images
* Overplot region files to review any sources you've found
* Plot the SNe.reg file to see if you recover the SNe we know are in the images
* Can you find any other sources?

### Questions to think about:

* How can would you improve the source detection step to remove false positives? Remember what you have at your disposal: 
   * Masks, error arrays, and SExtractor source properties
* Often times we only want to regard a source as real if it appears in more than one filter or epoch. How could you adapt this code to achieve that?
* What can you do with ML vs algorithmic source detection? How would you create a training set?
* If you were interested in "high-z" candidates, how would you screen for that? Remember you have a catalog of the field...

---

## Let's Look at Examples of "Good" and "Bad" Sources

### Good Sources:<br>

<img style="float: center;" src='./assets/GoodSource1.png' width="200px"/>
<img style="float: center;" src='./assets/GoodSource2.png' width="200px"/><br>

* PSF-like shapes (not too sharp, gaussian profile)
* Lack of dipoles (artifact caused by bad alignment)
* Noise properties around source are uniform

<hr style="border:1px solid gray"> </hr>

### Bad Sources:<br>

<img style="float: center;" src='./assets/BadSource_CoreResidual.png' width="200px"/>
<img style="float: center;" src='./assets/BadSource_CR.png' width="200px"/>
<img style="float: center;" src='./assets/BadSource_MaskingArtifact.png' width="200px"/><br>

* PSF-like shapes (too sharp, appear as hot pixels)
* Black and white core residuals (artifact caused by subtracting bright and complicated cores from each other [e.g., galaxies, diffraction spikes, etc])